In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.preprocessing import StandardScaler
from flask import Flask, request

#returns zip code from address
def get_zipcode(address):
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(address, exactly_one=True)

    if location:
        return location.address.split(",")[-2]#.raw#['address']#['municipality']
    else:
        return None
    
def info_CP(cp):
    pob = municipios.at[(municipios["Codigo Postal"] == cp).idxmax(),"Poblacion"]
    #remove non numeric characters from pob
    pob = int(''.join(filter(str.isdigit, pob)))
    #density in hab/m2 (shops)
    den = municipios.at[(municipios["Codigo Postal"] == cp).idxmax(),"Densidad Comercial"]
    rent = municipios.at[(municipios["Codigo Postal"] == cp).idxmax(),"Renta Por Hogar"]
    #rent is in €
    #remove non numeric characters from rent
    rent = int(''.join(filter(str.isdigit, rent)))
    
    return (pob, den, rent)
    
#read the csv file
shops = pd.read_csv("Fichero surtido tiendas 10062023.csv", sep=";")
#clean the dataframe
shops = shops.drop(columns=['COMUNIDAD','PROVINCIA','FECHA APERTURA'])
shops.rename(columns={'CV 2021':'CV_1'}, inplace=True)
shops.rename(columns={'CV 2022':'CV_2'}, inplace=True)
shops.rename(columns={'Nivel de surtido':'SURTIDO'}, inplace=True)
shops.rename(columns={'SUP M2':'M2'}, inplace=True)
shops.rename(columns={'C. POSTAL':'CP'}, inplace=True)

#get location data from zipcodes
municipios = pd.read_csv("Variables Geoblink Codigo Postal.csv", sep=";")
#remove lines with empty data
municipios = municipios.dropna()

shops["CP"] = shops.CP.apply(info_CP)

#Define a mapping of string values of TARIFA to numerical equivalents
mapping = {'AGRESIVA': 1, 'ESTANDAR': 2, 'PLUS': 3, 'XPLUS': 4}

#Apply the mapping to the TARIFA column
shops['TARIFA'] = shops['TARIFA'].map(mapping)

#Define a mapping of string values of ROL to numerical equivalents
mapping = {'CONVENIENCIA':1, 'URB.RESIDENCIAL': 3, 'URB.COMERCIAL': 2, 'RURAL': 4}

#Apply the mapping to the ROL column
shops['ROL'] = shops['ROL'].map(mapping)

#Define a mapping of string values of SURTIDO to numerical equivalents
mapping = {'NACIONAL3':1, 'NACIONAL4': 2, 'NACIONAL5': 3, 'NACIONAL6': 4, 'NACIONAL7': 5, 'NACIONAL8': 6}

#Apply the mapping to the SURTIDO column
shops['SURTIDO'] = shops['SURTIDO'].map(mapping)

#remove non numeric characters from CV_1 and CV_2
shops['CV_1'] = shops['CV_1'].str.replace("’",'')
shops['CV_1'] = shops['CV_1'].str.replace("€",'')
shops['CV_1'] = shops['CV_1'].str.replace(" ",'')

shops['CV_2'] = shops['CV_2'].str.replace("’",'')
shops['CV_2'] = shops['CV_2'].str.replace("€",'')
shops['CV_2'] = shops['CV_2'].str.replace(" ",'')

#transform CV_1 and CV_2 to numeric
shops['CV_1'] = pd.to_numeric(shops['CV_1'])
shops['CV_2'] = pd.to_numeric(shops['CV_2'])

#split the tuple into 3 columns
shops[['POBLACION','DENSIDAD','RENTA']] = pd.DataFrame(shops.CP.tolist(), index= shops.index)

shops = shops.drop(columns=['CV_1','MUNICIPIO','CP'])

# M2 en metros cuadrados de la tienda
# TARIFA: 1: Agresiva, 2: Estandar, 3: Plus, 4: XPlus
# ROL: 1: Conveniencia, 2: Urbana Comercial, 3: Urbana Residencial, 4: Rural
# SURTIDO: 1: Nacional 3, 2: Nacional 4, 3: Nacional 5, 4: Nacional 6, 5: Nacional 7, 6: Nacional 8
# CP: Codigo Postal
# tiendas: numero de tiendas a devolver
# Devuelve un dataframe con la tienda mas parecida a la tienda de entrada
def tienda_espejo_CP(M2,TARIFA,ROL,SURTIDO,CP, tiendas):
    features = shops.drop(['CV_2','TIENDA'], axis = 1)
    scaler_raw = StandardScaler()
    X_nn = scaler_raw.fit_transform(features)
    X = np.array([M2,TARIFA,ROL,SURTIDO,info_CP(CP)[0],info_CP(CP)[1],info_CP(CP)[2]])
    X = scaler_raw.transform(X.reshape(1, -1))
    #find nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=tiendas, algorithm='auto').fit(X_nn)
    result = nbrs.kneighbors(X,2,return_distance=False)
    
    # for i in range(tiendas):
    #     print((shops.iloc[result[0][i]]))
    s = shops[shops.index.isin(result[0])]
    return s

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# M2 en metros cuadrados de la tienda
# TARIFA: 1: Agresiva, 2: Estandar, 3: Plus, 4: XPlus
# ROL: 1: Conveniencia, 2: Urbana Comercial, 3: Urbana Residencial, 4: Rural
# SURTIDO: 1: Nacional 3, 2: Nacional 4, 3: Nacional 5, 4: Nacional 6, 5: Nacional 7, 6: Nacional 8
# CP: Codigo Postal
# tiendas: numero de tiendas a devolver
# Devuelve un dataframe con la tienda mas parecida a la tienda de entrada

print(tienda_espejo_CP(50,2,2,4,47001,2))

      M2             TIENDA  TARIFA  ROL  SURTIDO        CV_2  POBLACION  \
72   169  PXSE HARINAS, 7-9       2    2        4  1449857.55       9970   
286  117  PXSE ANTONIA DÍAZ       2    2        5   824735.47       9970   

     DENSIDAD  RENTA  
72      554.9  45150  
286     554.9  45150  


/Users/frrm/miniconda3/envs/sklearn-env/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
